In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2G
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split

In [ ]:
# image format is 200x200 age_gender_race_imagename.jpeg (22_1_123142424.jpeg)

In [ ]:
path = "C:\\Users\\sahay\\OneDrive\\Desktop\\Final_Assignment\\age_gender\\UTKFace"
images = []
age = []
gender = []
for img in os.listdir(path):
  ages = img.split("_")[0]
  genders = img.split("_")[1]
  img = cv2.imread(str(path)+"/"+str(img))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  images.append(np.array(img))
  age.append(np.array(ages))
  gender.append(np.array(genders))

In [ ]:
age = np.array(age,dtype=np.int64)
images = np.array(images)   
gender = np.array(gender,np.uint64)

In [ ]:
x_train_age, x_test_age, y_train_age, y_test_age = train_test_split(images, age,test_size=0.30, random_state=42)
x_train_gender, x_test_gender, y_train_gender, y_test_gender = train_test_split(images, gender,test_size=0.30, random_state=42)

In [ ]:
# Age model
age_model = Sequential()
age_model.add(Conv2D(32, kernel_size=3,padding='same', activation='relu', input_shape=(200,200,3)))
age_model.add(MaxPool2D(pool_size=3))

age_model.add(Conv2D(64, kernel_size=3,padding='same', activation='relu'))
age_model.add(MaxPool2D(pool_size=3))

age_model.add(Conv2D(128, kernel_size=3, padding='same',activation='relu'))
age_model.add(MaxPool2D(pool_size=3))

age_model.add(Conv2D(256, kernel_size=3, padding='same',activation='relu'))
age_model.add(MaxPool2D(pool_size=3))

age_model.add(Flatten())

age_model.add(Dropout(0.50))
age_model.add(Dense(128, activation='relu'))

age_model.add(Dense(1, activation='linear', name='age'))

In [ ]:
age_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
print(age_model.summary())    

In [ ]:
history_age = age_model.fit(x_train_age, y_train_age,validation_data=(x_test_age, y_test_age), epochs=60)

In [ ]:
age_model.save('age_model.h5')

In [ ]:
#gender model
gender_model = Sequential()

gender_model.add(Conv2D(36, kernel_size=3, activation='relu', input_shape=(200,200,3)))
gender_model.add(MaxPool2D(pool_size=3))

gender_model.add(Conv2D(64, kernel_size=3, activation='relu'))
gender_model.add(MaxPool2D(pool_size=3))

gender_model.add(Conv2D(128, kernel_size=3, activation='relu'))
gender_model.add(MaxPool2D(pool_size=3))

gender_model.add(Conv2D(256, kernel_size=3, activation='relu'))
gender_model.add(MaxPool2D(pool_size=3))

gender_model.add(Flatten())

gender_model.add(Dropout(0.2))
gender_model.add(Dense(512, activation='relu'))

gender_model.add(Dense(1, activation='sigmoid', name='gender'))

In [ ]:
gender_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_gender = gender_model.fit(x_train_gender, y_train_gender,validation_data=(x_test_gender, y_test_gender), epochs=50)

In [ ]:
gender_model.save('gender_model.h5')

In [ ]:
history = history_gender

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

In [ ]:
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [38]:
from keras.models import load_model

In [39]:
my_model = load_model('age_model.h5', compile=False)

In [40]:
predictions = my_model.predict(x_test_age)
y_pred = (predictions>= 0.5).astype(int)[:,0]

In [41]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test_age, y_pred))

Accuracy =  0.03163222269084774


In [ ]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm=confusion_matrix(y_test_gender, y_pred)  
sns.heatmap(cm, annot=True)